<a href="https://colab.research.google.com/github/kashyap333/Lazy_functions/blob/main/Lazy_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm

from torch import nn
import torch
import torchvision

import os
import zipfile

from pathlib import Path

import requests

import os

In [14]:
def Training(model: torch.nn.Module, train_loader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, accuracy_fn, optimizer: torch.optim.Optimizer):

  train_loss, train_acc = 0
  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  for batch, (X_train, y_train) in enumerate(train_loader):

    X_train, y_train, model = X_train.to(device), y_train.to(device), model.to(device)

    predictions = model(X_train)

    loss = loss_fn(predictions, y_train)
    train_loss += loss
    train_acc += accuracy_fn(y_traue = y_train, y_prediction = predictions.argmax(dim=1))

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()


  print(f'train loss / epoch = {(train_loss/len(train_loader)):.5f}\n train_accuracy / epoch = {(train_acc/len(train_loader)):.5f}')





In [13]:
def test_step(test_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn):
  
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    test_loss, test_acc = 0, 0
    model.eval() 

    with torch.inference_mode(): 
        for X_test, y_test in test_loader:
            # Send data to GPU
            X_test, y_test = X_test.to(device), y_test.to(device)
            
            # 1. Forward pass
            test_pred = model(X_test)
            
            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y_test)
            test_acc += accuracy_fn(y_true=y_test,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )
        
        # Adjust metrics and print out
        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [10]:
def accuracy_fn(y_true, predictions):

  correct = torch.eq(y_true, predictions).sum().item()
  acc = (correct / len(predictions)) * 100
  return acc